<a href="https://colab.research.google.com/github/r0bert8841/Fantasy-Hockey-Predictions/blob/main/Historical_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_player = pd.read_csv('/content/drive/MyDrive/hockey_stats/player_list', sep='|')
df_player=df_player.reset_index()

In [6]:
seasons = ['20102011','20112012','20122013','20132014','20142015','20152016','20162017','20172018','20182019','20192020','20202021','20212022' ,'20222023']
#seasons = ['19981999','19992000','20002001','20012002','20022003','20032004','20042005','20052006','20062007','20072008','20082009','20092010' ,'20232024']

In [7]:
df = pd.DataFrame() # Initialize an empty dataframe
for season in seasons: # Iterate through a list called seasons
  df_season = pd.read_csv('/content/drive/MyDrive/hockey_stats/game_logs/logs'+str(season), sep='|') # Read in a CSV file for the given season
  df=pd.concat([df,df_season],axis=0) # Concatenate the data for the current season into the main dataframe

<ipython-input-7-6901370be05d>:3: DtypeWarning: Columns (14,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_season = pd.read_csv('/content/drive/MyDrive/hockey_stats/game_logs/logs'+str(season), sep='|')
<ipython-input-7-6901370be05d>:3: DtypeWarning: Columns (14,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_season = pd.read_csv('/content/drive/MyDrive/hockey_stats/game_logs/logs'+str(season), sep='|')


In [14]:
# build_sample(df,200,20122013,['goals', 'assists','points', 'shifts'])
def build_sample(df,df_player,max_games,min_season,stat_cols):
  df=df.sort_values(by=['player_id','date']).reset_index()
  df_goals = df[['season','player_id','date']+stat_cols].dropna()

  df_goals['game_count'] = df_goals.groupby('player_id')['season'].rolling(2000, min_periods=1).count().reset_index(0,drop=True)
  df_goals['game_count_curr_season'] = df_goals.groupby(by=['player_id','season'])['date'].rolling(2000, min_periods=1).count().reset_index(0,drop=True).reset_index(0,drop=True)
  df_goals['min_season'] = df_goals.groupby('player_id')['season'].rolling(2000, min_periods=1).min().reset_index(0,drop=True)

  df_goals=df_goals.sort_values(by=['player_id','date'], ascending=False).reset_index()
  df_goals['max_games'] = df_goals.groupby('player_id')['game_count'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)

  df_goals=df_goals.sort_values(by=['player_id','date'], ascending=True).reset_index()
  df_sample = df_goals[(df_goals['max_games']>max_games) & (df_goals['min_season']>min_season)]

  df_sample = df_sample.merge(df_player[['id','season','type']].drop_duplicates(), left_on=['player_id','season'], right_on=['id','season'],suffixes=('','_pos'))
  #df_sample = df_sample.drop('id')

  return df_sample[['season','player_id','date']+stat_cols+['game_count','game_count_curr_season','min_season','max_games','type']]

In [29]:
def build_features(df,stat_col):

  df[stat_col+'_mean_career'] = df.groupby('player_id')[stat_col].rolling(2000, min_periods=1).mean().reset_index(0,drop=True)
  df[stat_col+'_mean_82'] = df.groupby('player_id')[stat_col].rolling(82, min_periods=1).mean().reset_index(0,drop=True)
  #df_goals['goals_mean_41'] = df_goals.groupby('player_id')['goals'].rolling(41, min_periods=1).mean().reset_index(0,drop=True)
  #df_goals['goals_mean_20'] = df_goals.groupby('player_id')['goals'].rolling(20, min_periods=1).mean().reset_index(0,drop=True)

  #df_sample = df_sample[['player_id','game_count','goals_mean_career','goals_mean_82']]
  #df_groupby = df.groupby('game_count')['goals_mean_career'].mean().reset_index()
  #df_sample_mean = df_goals[df_goals['min_season']>20122013].merge(df_goals_mean_career_mean, left_on='game_count', right_on='game_count',suffixes=('','_mean'))


  # Do I need to do this filter?
  #df=df[df['game_count']>81].sort_values(by=['player_id','date'], ascending=True)#.reset_index()
  df=df.sort_values(by=['player_id','date'], ascending=True)#.reset_index()
  df[stat_col+'_mean_career_max'] = df.groupby('player_id')[stat_col+'_mean_career'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)
  df[stat_col+'_mean_82_max'] = df.groupby('player_id')[stat_col+'_mean_82'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)

  df[stat_col+'_mean_career_max_lag'] = df[stat_col+'_mean_career_max'].shift(1)
  df[stat_col+'_mean_82_max_lag'] = df[stat_col+'_mean_82_max'].shift(1)
  df[stat_col+'_mean_82_max_change'] = df[stat_col+'_mean_82_max_lag'] != df[stat_col+'_mean_82_max']
  df[stat_col+'_mean_career_max_change'] = df[stat_col+'_mean_career_max_lag'] != df[stat_col+'_mean_career_max']

  df['games_count_'+stat_col+'_mean_82_max'] = df[stat_col+'_mean_82_max_change'] * df['game_count']
  df['games_count_'+stat_col+'_mean_career_max'] = df[stat_col+'_mean_career_max_change'] * df['game_count']

  df['games_count_'+stat_col+'_mean_career_max'] = df.groupby('player_id')['games_count_'+stat_col+'_mean_career_max'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)
  df['games_count_'+stat_col+'_mean_82_max'] = df.groupby('player_id')['games_count_'+stat_col+'_mean_82_max'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)

  df['games_since_'+stat_col+'_mean_career_max'] = df['game_count'] - df['games_count_'+stat_col+'_mean_career_max']
  df['games_since_'+stat_col+'_mean_82_max'] = df['game_count'] - df['games_count_'+stat_col+'_mean_82_max']

  #This is to get the predictor 82 games in the future
  df = df.sort_values(by=['player_id','date'], ascending=False)

  #df['pred'+stat_col+'_mean_career_max'] = df[stat_col+'_mean_career'].shift(82) # This doesn't make sense as a predictor
  df['pred_'+stat_col+'_mean_82'] =df.groupby('player_id')[stat_col+'_mean_82'].shift(82) # df[stat_col+'_mean_82'].shift(82)
  df['pred_'+stat_col+'_mean_82_abs_diff'] = (df['pred_'+stat_col+'_mean_82'] - df[stat_col+'_mean_82']).abs()
  df['pred_'+stat_col+'_mean_82_new_max'] = df['pred_'+stat_col+'_mean_82'] > df[stat_col+'_mean_82_max']
  df['pred_'+stat_col+'_mean_82_increase'] = df['pred_'+stat_col+'_mean_82'] > df[stat_col+'_mean_82']
  df = df.sort_values(by=['player_id','date'], ascending=True)

  df = df.drop(columns=[stat_col,stat_col+'_mean_82_max_change',stat_col+'_mean_career_max_change',stat_col+'_mean_career_max_lag',stat_col+'_mean_82_max_lag','games_count_'+stat_col+'_mean_career_max','games_count_'+stat_col+'_mean_82_max'])
  #df = df.drop(columns=[stat_col,stat_col+'_mean_82_max_change',stat_col+'_mean_82_max_lag','games_count_'+stat_col+'_mean_82_max'])
  #df = df.drop(columns=['season','date','min_season','max_games'])
  # Add games since max
  # Add min values and games since min
  # Add games since team change
  # games/days since Start of the season? or maybe some sort of moving average for time between games.
  # add predictor columns ( the mean_82 in 82 games ) (and variance as a predictor between 82 mean and the future 82 mean)
  # maybe add game counter for the current season
  # add league average of scoring over the last 82 games and add team average of scoring over the last 82 games so that when someone changes teams the team value will change too.


  # Here we are mapping out the analytics we were using to try and map out players career stages
  # Only want to apply this after game_count 81
  df_rookie = df[df['game_count']<=81]
  df_rookie[stat_col+'_mean_career_var'] = np.nan
  df_rookie[stat_col+'_mean_82_var'] = np.nan
  df_rookie[stat_col+'_mean_diff'] = np.nan
  df_rookie[stat_col+'_mean_diff_perc'] = np.nan
  df_rookie[stat_col+'_mean_diff_perc_rolling_min'] = np.nan
  df_rookie[stat_col+'_mean_diff_perc_rolling_max'] = np.nan
  df_rookie[stat_col+'_game_count_min'] = np.nan
  df_rookie[stat_col+'_game_count_max'] = np.nan
  df_rookie[stat_col+'_slope_since_min'] = np.nan
  df_rookie[stat_col+'_slope_since_max'] = np.nan
  df_rookie[stat_col+'_shift_41_games'] = np.nan



  df_established = df[df['game_count']>81]
  # I want to create a column for running variance of certain columns, I think this can be a solid gauge
  # Calculate the rolling variance of career mean points for each player
  df_established[stat_col+'_mean_career_var'] = df_established.groupby('player_id')[stat_col+'_mean_career'].rolling(2000, min_periods=1).var().reset_index(0,drop=True)
  # Calculate the rolling variance of 82-game mean points for each player
  df_established[stat_col+'_mean_82_var'] = df_established.groupby('player_id')[stat_col+'_mean_82'].rolling(2000, min_periods=1).var().reset_index(0,drop=True)
  # Calculate the difference between 82-game mean points and career mean points for each player
  df_established[stat_col+'_mean_diff'] = df_established[stat_col+'_mean_82'] - df_established[stat_col+'_mean_career']
  # Calculate the percentage difference between 82-game mean points and career mean points for each player
  df_established[stat_col+'_mean_diff_perc'] = np.where(
      df_established[stat_col+'_mean_career'] != 0,
      (df_established[stat_col+'_mean_82'] - df_established[stat_col+'_mean_career']) / df_established[stat_col+'_mean_career'],
      np.nan  # Set to NaN when 'points_mean_career' is zero
  )

  # I want to create a column for running variance of certain columns, I think this can be a solid gauge
  df_established[stat_col+'_mean_diff_perc_rolling_min'] = df_established.groupby('player_id')[stat_col+'_mean_diff_perc'].rolling(2000, min_periods=1).min().reset_index(0,drop=True)
  df_established[stat_col+'_mean_diff_perc_rolling_max'] = df_established.groupby('player_id')[stat_col+'_mean_diff_perc'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)



  # prompt: can you write a variable game_count_min which is game_count*(points_mean_diff_perc_rolling_min == points_mean_diff_perc)

  df_established[stat_col+'_game_count_min_0'] = df_established['game_count'] * (df_established[stat_col+'_mean_diff_perc_rolling_min'] == df_established[stat_col+'_mean_diff_perc'])
  df_established[stat_col+'_game_count_max_0'] = df_established['game_count'] * (df_established[stat_col+'_mean_diff_perc_rolling_max'] == df_established[stat_col+'_mean_diff_perc'])
  #both of these rolling functions should be max to get the most up to date min and max
  df_established[stat_col+'_game_count_min'] = df_established.groupby('player_id')[stat_col+'_game_count_min_0'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)
  df_established[stat_col+'_game_count_max'] = df_established.groupby('player_id')[stat_col+'_game_count_max_0'].rolling(2000, min_periods=1).max().reset_index(0,drop=True)
  df_established.drop(columns=[stat_col+'_game_count_min_0',stat_col+'_game_count_max_0'], inplace=True)

  df_established[stat_col+'_slope_since_min'] = (df_established[stat_col+'_mean_diff_perc']-df_established[stat_col+'_mean_diff_perc_rolling_min'])/(df_established['game_count']-df_established[stat_col+'_game_count_min'])
  df_established[stat_col+'_slope_since_max'] = (df_established[stat_col+'_mean_diff_perc']-df_established[stat_col+'_mean_diff_perc_rolling_max'])/(df_established['game_count']-df_established[stat_col+'_game_count_max'])
  df_established[stat_col+'_slope_since_min'] = df_established[stat_col+'_slope_since_min'].fillna(0)
  df_established[stat_col+'_slope_since_max'] = df_established[stat_col+'_slope_since_max'].fillna(0)

  df_established[stat_col+'_shift_41_games'] = df_established.groupby('player_id')[stat_col+'_mean_diff_perc'].shift(41)
  df_established[stat_col+'_slope_41_games'] = (df_established[stat_col+'_mean_diff_perc']-df_established[stat_col+'_shift_41_games'])/41
  df_established[stat_col+'_slope_41_games'] = df_established[stat_col+'_slope_41_games'].fillna(0)
  df_established.drop(columns=[stat_col+'_shift_41_games'], inplace=True)

  # Concatenate df_rookie and df_established vertically
  df_all = pd.concat([df_rookie, df_established], axis=0)


  return df_all

In [30]:
df_test = build_sample(df,df_player,200,20122013,['goals', 'assists','points'])

In [31]:
df1 = build_features(df_test,'goals')
df2 = build_features(df1,'assists')
df3 = build_features(df2,'points')
#df4 = build_features(df3,'shifts')

<ipython-input-29-89b35a527496>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rookie[stat_col+'_mean_career_var'] = np.nan
<ipython-input-29-89b35a527496>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rookie[stat_col+'_mean_82_var'] = np.nan
<ipython-input-29-89b35a527496>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [34]:
# prompt: can you write df4 to my google drive
df3.to_csv('/content/drive/MyDrive/hockey_stats/features.csv', sep='|')
